In [1]:
import os
import pandas as pd
import datetime
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

In [2]:
df = pd.DataFrame()
for f in [x for x in os.listdir("data") if x[2] == "_"]:
    df = pd.concat([df, pd.read_parquet(os.path.join("data",f))])

In [3]:
df['oscrapovano_minuty'] = df['oscrapovano'].apply(lambda x: str(x)[0:14])

In [4]:
df['oscrapovano_minuty']

0      2024-11-07 18:
1      2024-11-07 18:
2      2024-11-07 18:
3      2024-11-07 18:
4      2024-11-07 18:
            ...      
516    2025-05-01 19:
517    2025-05-01 19:
518    2025-05-01 10:
519    2025-05-01 10:
520    2025-05-01 10:
Name: oscrapovano_minuty, Length: 8474571, dtype: object

In [5]:
df = df.drop_duplicates(subset=['odkud','kam','odjezd','oscrapovano_minuty'], keep="last")

In [6]:
len(df)

3445820

In [7]:
df[['odjezd','odkud','kam']].drop_duplicates().shape[0]

216433

In [8]:
df.groupby("prodejce")['oscrapovano'].max()

prodejce
ARR   2025-05-01 22:36:00
LE    2025-05-01 22:27:45
RJ    2025-05-01 22:17:15
ČD    2025-05-01 22:12:56
Name: oscrapovano, dtype: datetime64[ns]

In [9]:
df.groupby(['prodejce',pd.Grouper(key='oscrapovano',freq='D')]).size()

prodejce  oscrapovano
ARR       2024-11-07      2297
          2024-11-08      7128
          2024-11-09      3320
          2024-11-10      3157
          2024-11-11      3367
                         ...  
ČD        2025-04-27     10326
          2025-04-28     11846
          2025-04-29     11224
          2025-04-30     11575
          2025-05-01     10899
Length: 704, dtype: int64

Výpadek ČD 2024-11-26 opraven, vyřešen.

In [11]:
df = df.sort_values(by="oscrapovano").reset_index(drop=True)

In [12]:
df = df.dropna(subset=['odkud','kam','odjezd','oscrapovano'],how='any')

In [13]:
df = df[df['prostredek'] != 'autobus']

In [14]:
df = df[~df['kam'].str.contains('(tief)',na=False)]
df = df[~df['odkud'].str.contains('(tief)',na=False)]

C:\Users\micha\AppData\Local\Temp\ipykernel_30376\103445691.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[~df['kam'].str.contains('(tief)',na=False)]
C:\Users\micha\AppData\Local\Temp\ipykernel_30376\103445691.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[~df['odkud'].str.contains('(tief)',na=False)]


Před 7. listopadem scrapování testovací a nesystematické = nevypovídající = filtrujeme.

In [16]:
df = df[df['oscrapovano'] >= '2024-11-08']

In [17]:
df.groupby(['prodejce',pd.Grouper(key='oscrapovano',freq='D')]).size()

prodejce  oscrapovano
ARR       2024-11-08      7128
          2024-11-09      3320
          2024-11-10      3157
          2024-11-11      3367
          2024-11-12      3323
                         ...  
ČD        2025-04-27      9895
          2025-04-28     10607
          2025-04-29     10361
          2025-04-30     10570
          2025-05-01     10236
Length: 692, dtype: int64

In [18]:
days = {0: 'po', 1: 'út', 2: 'st', 3: 'čt', 
        4: 'pá', 5: 'so', 6: 'ne'}
df['den'] = df['odjezd'].dt.dayofweek.map(days)

In [19]:
df['predstih_d'] = df['predstih'].dt.days
df['predstih_h'] = df['predstih'].dt.total_seconds() / 3600

In [20]:
df = df[df['predstih_h'] > -3]

In [21]:
kategoricka_data = ['odkud','kam','prodejce','den']

In [22]:
for k in kategoricka_data:
    print("Before:", df[k].memory_usage(deep=True))
    df[k] = df[k].astype('category')
    print("After: ", df[k].memory_usage(deep=True))

Before: 215505697
After:  27420289
Before: 215515767
After:  27415476
Before: 231572555
After:  27406885
Before: 201793355
After:  27407205


In [23]:
df['cena'] = pd.to_numeric(df['cena'])

In [24]:
df.shape[0]

3045165

In [25]:
df[['odjezd','odkud','kam']].drop_duplicates().shape[0]

192232

In [26]:
poradi = ['oscrapovano','prodejce','odkud','kam',
 'odjezd',
 'predstih',
 'predstih_d',
 'predstih_h',
          'cena',
 'prostredek',
          'volnych_mist',
 'obsazenost',
 'jizdni_doba',
 'vzdalenost',
 'zpozdeni',
 'cena_poznamka',
 'den',
 'prestupy',
'vlaky',
 'mistenka_zdarma',
 'nahradni_bus',
 'volna_mista_economy',
 'volna_mista_economy_plus',
 'volna_mista_economy_business',
 'volna_mista_premium']

In [27]:
df[poradi].to_parquet(os.path.join("data","jizdenky.parquet"))

In [28]:
nejnovejsi = df['oscrapovano'].max()
nejnovejsi
df_tyden = df[df['oscrapovano'] > (nejnovejsi - datetime.timedelta(hours=168))]

In [29]:
df_tyden[poradi].to_csv(os.path.join("data","jizdenky_tyden.csv"))